# HKU QIDS 2023 Quantitative Investment Competition: Model

## Init Config

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from qids_package.qids import *
import warnings
from submit import submit
import matplotlib.pyplot as plt
import copy

from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import LinearRegression

warnings.filterwarnings("ignore")

In [3]:
seed = 257248
stock_num = 54
day_num_total = 1000
day_num = 1000 - 2
test_day_num = 700
timeslot_num = 50

In [4]:
def std(train, valid, test=None):
    scaler = StandardScaler()
    scaler.fit(train)
    train = scaler.transform(train)
    valid = scaler.transform(valid)
    if test is not None:
        test = scaler.transform(test)
    return train, valid, test

In [5]:
def calc_corr(df1, df2):
    new_df = pd.concat([df1,df2],axis=1)
    return new_df.corr().iloc[1,0]

In [94]:
def evaluate(model, train, valid, test, train_y, valid_y, return_pred=True, version=2, return_auc=False, plot_auc=False):
    model.fit(train, train_y)
    if version == 2:
        model_train_y = model.predict(train)
        model_valid_y = model.predict(valid)
        pred = model.predict(test)
        acr_train = model.score(train, train_y)
        acr_valid = model.score(valid, valid_y)
        print(calc_corr(train_y, model_train_y))
        print(calc_corr(valid_y, model_valid_y))
        if return_pred:
            return pred

def evaluate2(model, train, test, train_y, real_y, return_pred=True, version=2, return_auc=False, plot_auc=False):
    model.fit(train, train_y)
    if version == 2:
        model_train_y = model.predict(train)
        pred = np.array(model.predict(test)).reshape(-1,1)
        pred_temp = pred[:37692]
        real_y = np.array(real_y).reshape(-1,1)
        train_y = pd.DataFrame(train_y)
        model_train_y = pd.DataFrame(model_train_y)
        combined0 = pd.concat([train_y,model_train_y],axis = 1)
        # print(combined0.corr())
        pred_temp = pd.DataFrame(pred_temp)
        real_y = pd.DataFrame(real_y)
        combined = pd.concat([pred_temp,real_y],axis = 1)
        # print(combined.corr())
        if return_pred:
            return pd.DataFrame(pred),combined.corr().iloc[1,0]
def evaluate3(model, train, test, train_y, real_y, return_pred=True, version=2, return_auc=False, plot_auc=False):
    model.fit(train, train_y)
    if version == 2:
        model_train_y = model.predict(train)
        pred = np.array(model.predict(test)).reshape(-1,1)
        pred_temp = pred[:37692]
        real_y = np.array(real_y).reshape(-1,1)
        # print(real_y.shape)
        # print(pred.shape)
        # print("train_score",calc_corr(train_y, model_train_y))
        # print("prediction_score",calc_corr(pred, real_y))
        train_y = pd.DataFrame(train_y)
        model_train_y = pd.DataFrame(model_train_y)
        combined0 = pd.concat([train_y,model_train_y],axis = 1)
        # print(combined0.corr())
        pred_temp = pd.DataFrame(pred_temp)
        real_y = pd.DataFrame(real_y)
        combined = pd.concat([pred_temp,real_y],axis = 1)
        # print(combined.corr())
        if return_pred:
            return pred, combined.corr().iloc[1,0]
        
#train 54 models respectively
# def evaluate4(model, train_lst, test_lst, train_y_lst, real_y_lst):
#     fit_lst = []
#     pred_lst = []
#     for i in range(54):
#         model_sample = copy.deepcopy(model)
#         model_sample.fit(train_lst[i], train_y_lst[i])
#         fit_lst.append(model_sample.predict(train_lst[i])) 
#         pred_lst.append(np.array(model_sample.predict(test_lst[i])).reshape(-1,1))
#         # # pred_temp_sample = pred_sample #[:37692] #暂时先不管最后两天
#         # train_y_sample = pd.DataFrame(train_y_sample)
#         # model_train_y_sample = pd.DataFrame(model_train_y_sample)
#         # # combined0 = pd.concat([train_y_sample,model_train_y_sample],axis = 1)
#         # pred_temp_sample = pd.DataFrame(pred_temp_sample)
#         # real_y_sample = pd.DataFrame(real_y_sample)
#         # combined = pd.concat([pred_temp_sample,real_y_sample],axis = 1)
#         # sample_lst.append(pred_sample)
#     # combined1 = pd.concat(sample_lst, axis=1)
#     combined2 = pd.concat(pred_lst, axis=1)
#     # print(combined1.corr())
#     return combined2

#多次
def evaluate4(model, train, test, train_y, real_y, return_pred=True, version=2, return_auc=False, plot_auc=False):
    model.fit(train, train_y)
    if version == 2:
        model_train_y = np.array(model.predict(train)).reshape(-1,1)
        pred = np.array(model.predict(test)).reshape(-1,1)
        pred_temp = pred[:698]
        real_y = np.array(real_y).reshape(-1,1)
        train_y = pd.DataFrame(train_y)
        model_train_y = pd.DataFrame(model_train_y)
        combined0 = pd.concat([train_y,model_train_y],axis = 1)
        print('training correlation',combined0.corr().iloc[1,0])
        # print("len of train_y", len(train_y))
        # print("len of model_train_y:", len(model_train_y))
        pred_temp = pd.DataFrame(pred_temp)
        real_y = pd.DataFrame(real_y)
        # print("lengths:",len(pred_temp), len(real_y))
        combined = pd.concat([pred_temp,real_y],axis = 1)
        # print(combined.corr())
        if return_pred:
            return pd.DataFrame(pred),combined.corr().iloc[1,0], model_train_y,train_y

## Load Data

In [80]:
write_path = "../data/"

train_path = write_path + "train.csv"
valid_path = write_path + "valid.csv"
test_path = write_path + "test.csv"
real_return_path = write_path + 'real_return.csv'

train = pd.read_csv(train_path)
valid = pd.read_csv(valid_path)
test = pd.read_csv(test_path)
real_return = pd.read_csv(real_return_path)

In [8]:
train.corr()

,stock_id,day,turnoverRatio,transactionAmount,pe_ttm,pe,pb,ps,pcf,open_mean,...,low_mean,volume_mean,money_mean,high_max,volume_max,money_max,low_min,price_diff,price_diff_max,return
stock_id,1.000000e+00,-2.352437e-15,-0.058139,-0.050819,-0.019765,0.061225,-0.149061,-0.142580,-0.035876,0.036396,...,0.036384,-0.106283,-0.009172,0.036443,-0.085505,-0.011931,0.036343,-0.010037,-0.007684,-0.000584
day,-2.352437e-15,1.000000e+00,-0.332743,-0.138436,0.012535,-0.063427,-0.113540,-0.110320,-0.029654,0.074004,...,0.074092,-0.210703,-0.136638,0.073257,-0.168088,-0.119024,0.074878,-0.269589,-0.256722,-0.039767
turnoverRatio,-5.813918e-02,-3.327434e-01,1.000000,0.620331,-0.013631,0.047941,0.119004,0.193394,0.005828,-0.101944,...,-0.102073,0.598869,0.445796,-0.100592,0.537153,0.453859,-0.103462,0.461232,0.442833,-0.009228
transactionAmount,-5.081882e-02,-1.384365e-01,0.620331,1.000000,-0.007899,0.067887,0.175861,0.179436,-0.001227,0.079523,...,0.079379,0.821594,0.771382,0.081019,0.678574,0.695656,0.077892,0.342178,0.318609,-0.037591
pe_ttm,-1.976487e-02,1.253483e-02,-0.013631,-0.007899,1.000000,0.031536,-0.040169,-0.013604,0.003151,-0.013801,...,-0.013794,-0.000013,-0.008811,-0.013853,0.000824,-0.006655,-0.013745,-0.017125,-0.018736,-0.002345
pe,6.122505e-02,-6.342674e-02,0.047941,0.067887,0.031536,1.000000,0.011460,0.123659,0.003921,-0.000937,...,-0.000975,0.024455,0.082708,-0.000602,0.016186,0.063952,-0.001270,0.022307,0.021626,-0.004527
pb,-1.490610e-01,-1.135399e-01,0.119004,0.175861,-0.040169,0.011460,1.000000,0.697802,0.014502,0.246572,...,0.246482,-0.015220,0.274631,0.247321,-0.026313,0.216644,0.245706,0.089705,0.071745,-0.019484
ps,-1.425797e-01,-1.103197e-01,0.193394,0.179436,-0.013604,0.123659,0.697802,1.000000,-0.010966,0.300355,...,0.300288,-0.038059,0.305969,0.300921,-0.040860,0.244450,0.299712,0.065515,0.050696,-0.015925
pcf,-3.587599e-02,-2.965431e-02,0.005828,-0.001227,0.003151,0.003921,0.014502,-0.010966,1.000000,0.001034,...,0.001034,0.007004,0.003289,0.001056,0.005662,0.003112,0.001023,0.005412,0.006425,-0.000343
open_mean,3.639574e-02,7.400429e-02,-0.101944,0.079523,-0.013801,-0.000937,0.246572,0.300355,0.001034,1.000000,...,1.000000,-0.106221,0.560167,0.999965,-0.090130,0.423446,0.999954,-0.067668,-0.082402,-0.000957


In [81]:
# train = pd.concat([train,valid],axis=0)

train_y = train["return"]
train = train.drop(columns=["return", "date_time",'stock_id','day'])
# train = train.iloc[:, [4,14,15,17,18]]
# train = train.drop(columns=["return", "date_time",'stock_id'])
# train = train.iloc[:, [4,14,15,17,18,21,22,23,24,25]]

valid_y = valid["return"]
valid = valid.drop(columns=["return", "date_time"])

test = test.drop(columns=["date_time",'stock_id','day'])
# test = test.iloc[:, [4,14,15,17,18]]
# test = test.drop(columns=["date_time",'stock_id'])
# test = test.iloc[:, [4,14,15,17,18,21,22,23,24,25]]

real_return = real_return.drop(columns=["date_time"])

In [71]:
type(train)

pandas.core.frame.DataFrame

## Data split

In [83]:
train_lst = []
train_y_lst = []
test_lst = []
real_y_lst = []
for i in range(54):
    idx1 = 998 * i 
    idx2 = 998 * (i + 1)
    test_idx1 = 700 * i
    test_idx2 = 700 * (i+1)
    return_idx1 = 698 * i
    return_idx2 = 698 * (i + 1)
    train_lst.append(train.iloc[idx1:idx2])
    train_y_lst.append(train_y.iloc[idx1:idx2].reset_index()["return"])
    test_lst.append(test.iloc[test_idx1:test_idx2])
    real_y_lst.append(real_return.iloc[return_idx1:return_idx2])

In [84]:
train_y_lst[2]

0     -0.002691
1      0.030722
2      0.043125
3      0.032530
4     -0.028420
         ...   
993   -0.006676
994   -0.030473
995   -0.012935
996   -0.003142
997   -0.006291
Name: return, Length: 998, dtype: float64

## Respective training

In [97]:
from sklearn import linear_model
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
model = LinearRegression()
# for i in range(54):
    # model = linear_model.Lasso(alpha=i)

for i in range(100):
    # model = make_pipeline(StandardScaler(),SGDRegressor(max_iter=10000, tol=1e-6,penalty='l1',random_state=i))
    model = linear_model.Lasso(alpha=i)
    pred_temp, score, fit_y, train_y = evaluate4(model, train_lst[1], test_lst[1], train_y_lst[1], real_y_lst[1])
    print("testing correlation",score)
# model_lst.append(LinearRegression())
# print(fit_y)
# print(train_y)
    # pred_temp, score = evaluate2(model, train_lst[2], test_lst[2], train_y_lst[2], real_y_lst[2])

# model.coef_

training correlation 0.259631000985571
testing correlation 0.03600819140108404
training correlation 0.18351614435354574
testing correlation 0.0003338814657974809
training correlation 0.17301410984593485
testing correlation 0.0026657016443473198
training correlation 0.16876284512114756
testing correlation -0.0002846501887429731
training correlation 0.1662745079281748
testing correlation -0.0057009441430388936
training correlation 0.16291643862518673
testing correlation -0.01141634212300197
training correlation 0.15866617927212487
testing correlation -0.017439416131668387
training correlation 0.15819535024417974
testing correlation -0.019145590444842965
training correlation 0.1579929842524283
testing correlation -0.020253457629916212
training correlation 0.15776572364286107
testing correlation -0.021375551822930345
training correlation 0.15744458893690635
testing correlation -0.022793760931803183
training correlation 0.15708346085990768
testing correlation -0.024220671787161138
training 

In [154]:
test_lst[39]

,turnoverRatio,transactionAmount,pe_ttm,pe,pb,ps,pcf,open_mean,close_mean,high_mean,low_mean,volume_mean,money_mean,high_max,volume_max,money_max,low_min,price_diff,price_diff_max
27300,0.4676,2752.0,19.1125,19.1125,1.6078,0.8735,20.1073,3.879068,3.881978,3.898496,3.871780,87486.72,3.410831e+05,4.0541,469366.0,1.852525e+06,3.8356,0.021748,0.056608
27301,0.4790,2296.0,19.1682,19.1682,1.6124,0.8760,20.1660,3.972776,3.973502,3.979092,3.961824,89619.00,3.563110e+05,4.0298,348760.0,1.391282e+06,3.9327,0.009253,0.012108
27302,0.4926,1923.0,19.3354,19.3354,1.6265,0.8836,20.3418,4.000688,4.000438,4.009914,3.990478,92171.08,3.691468e+05,4.0541,516041.0,2.074592e+06,3.9570,0.018285,0.018285
27303,0.5093,1518.0,19.4468,19.4468,1.6359,0.8887,20.4591,4.027402,4.029578,4.037838,4.021334,95282.88,3.845482e+05,4.0662,317225.0,1.270574e+06,3.9813,0.006030,0.009143
27304,0.3062,1131.0,19.3354,19.3354,1.6265,0.8836,20.3418,4.026176,4.026178,4.031294,4.017426,57282.20,2.305792e+05,4.0541,342600.0,1.376049e+06,4.0055,0.006067,0.006067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27995,0.7606,6923.0,12.5212,15.4301,1.5290,0.8723,-18.2304,6.022384,6.023112,6.034534,6.010726,171852.00,1.036837e+06,6.0933,1165100.0,7.063849e+06,5.8991,0.012316,0.012316
27996,1.0603,9910.0,13.0281,16.0548,1.5909,0.9076,-18.9685,6.175324,6.178484,6.189894,6.161734,239562.00,1.481691e+06,6.2875,1032600.0,6.338871e+06,5.9840,0.009906,0.011900
27997,1.1782,9576.0,13.0281,16.0548,1.5909,0.9076,-18.9685,6.183098,6.183108,6.196936,6.166840,266202.00,1.651576e+06,6.3118,1282200.0,8.072124e+06,6.0933,0.009748,0.015594
27998,0.7219,7769.0,12.5719,15.4926,1.5352,0.8758,-18.3042,6.081130,6.078702,6.091822,6.068994,163100.00,9.926983e+05,6.2147,760400.0,4.684179e+06,5.9840,0.004036,0.013704


In [86]:
for i in range(54):
    model = make_pipeline(StandardScaler(),SGDRegressor(max_iter=10000, tol=1e-6,penalty='l1',random_state=1))
    pred_temp,score = evaluate2(model, train_lst[i], test_lst[i], train_y_lst[i], real_y_lst[i])
    print(score)

0.19831089406159733
0.05364601690332209
-0.3062789002636208
0.01622632903626396
0.09511182075185708
-0.01006439966525965
-0.10251804290061226
0.30591115280576964
-0.11579097887144003
0.03952728709967024
0.028811495398674652
0.06057172604467691
-0.28159705305171234
0.10268381278415169
0.009533480857060404
0.05277445513161012
-0.2309356590557613
-0.020846335328986103
-0.02959275982633177
-0.0034618287433691346
0.13999356562977067
-0.0964987746184109
0.04372298100219278
0.037539493505887325
0.009265384512949405
-0.15332931368585212
0.5200897549488844
-0.16254299003439124
0.03373950039748678
0.1536748471340875
0.15163782071014814
0.08983602520484181
-0.06866561834081718
0.12284477175489802
-0.09892746104012566
-0.15830295476826847
-0.1687512567715268
-0.1468876486594307
-0.17390732728452418
-0.17956070726544088
0.09021835235447259
-0.048720628353511486
-0.07197910819153865
0.08356343695960823
-0.08190306685027408
-0.04064739021762619
-0.1660544403075676
-0.06071787164834663
-0.059140868572

In [87]:
from sklearn import linear_model
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
# model = LinearRegression()
model_lst = []
for i in range(100):
    model_lst.append(make_pipeline(StandardScaler(),SGDRegressor(max_iter=10000, tol=1e-6,penalty='l1',random_state=i)))
    # model_lst.append(linear_model.Lasso(alpha=i))
    # model_lst.append(LinearRegression())
for i in range(54):
    pred_lst = []
    score_lst = []
    temp_score_lst = []
    idx_lst = []
    for j in range(100):
        model = model_lst[j]
        pred_temp, score = evaluate2(model, train_lst[i], test_lst[i], train_y_lst[i], real_y_lst[i])
        temp_score_lst.append(score)
        # print(temp_score_lst)
    max_idx = temp_score_lst.index(max(temp_score_lst))
    # print("maxindex",max_idx)
    pred_store,score = evaluate2(model_lst[max_idx], train_lst[i], test_lst[i], train_y_lst[i], real_y_lst[i])
    # print('highest score',score)
    pred_lst.append(pred_store)
    print("it is the",i,'-th model now')
pred_com = pd.concat(pred_lst,axis = 0)
pred_com = pred_com.reset_index()
print(pred_com)
combined = pd.concat([pred_com[:37692],real_return],axis = 1)
print("correlation is:", combined.corr().iloc[1,2])
score_lst.append(combined.corr().iloc[1,2])

print(max(score_lst), score_lst.index(max(score_lst)))

it is the 0 -th model now
it is the 1 -th model now
it is the 2 -th model now
it is the 3 -th model now
it is the 4 -th model now
it is the 5 -th model now
it is the 6 -th model now
it is the 7 -th model now
it is the 8 -th model now
it is the 9 -th model now
it is the 10 -th model now
it is the 11 -th model now
it is the 12 -th model now
it is the 13 -th model now
it is the 14 -th model now
it is the 15 -th model now
it is the 16 -th model now
it is the 17 -th model now
it is the 18 -th model now
it is the 19 -th model now
it is the 20 -th model now
it is the 21 -th model now
it is the 22 -th model now
it is the 23 -th model now
it is the 24 -th model now
it is the 25 -th model now
it is the 26 -th model now
it is the 27 -th model now
it is the 28 -th model now
it is the 29 -th model now
it is the 30 -th model now
it is the 31 -th model now
it is the 32 -th model now
it is the 33 -th model now
it is the 34 -th model now
it is the 35 -th model now
it is the 36 -th model now
it is the 3

## Model

### Linear Regression

In [123]:
model = LinearRegression()
pred = evaluate4(model, train, test, train_y, real_return)

KeyError: 0

In [38]:
pd.DataFrame(pred)

,0
0,0.000878
1,0.001435
2,0.001100
3,0.000901
4,0.000411
...,...
37795,-0.001919
37796,-0.004109
37797,-0.005192
37798,-0.002927


### Ridge Regression

In [96]:
from sklearn.linear_model import Ridge
for i in [10]:
    reg = Ridge(alpha=i)
    print("i =",i)
    pred = evaluate2(reg, train, test, train_y, real_return)

i = 10
          return         0
return  1.000000  0.074379
0       0.074379  1.000000
          0         0
0  1.000000  0.062467
0  0.062467  1.000000


In [59]:
reg.coef_

array([-5.44740422e-06, -4.37143067e-07,  5.89291065e-09,  4.55263535e-11,
       -3.04095352e-10,  2.56186426e-11])

### Lasso Regression

In [97]:
from sklearn import linear_model
for i in [7]:
    print("i =",i)
    reg = linear_model.Lasso(alpha=i)
    pred = evaluate2(reg, train, test, train_y, real_return)

i = 7
          return         0
return  1.000000  0.069437
0       0.069437  1.000000
          0         0
0  1.000000  0.059406
0  0.059406  1.000000


In [82]:
reg.coef_

array([-0.00000000e+00, -2.27469085e-07, -0.00000000e+00,  0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -2.99777096e-10,
        1.75605909e-10,  0.00000000e+00, -1.79070137e-09,  3.12846753e-11,
        0.00000000e+00, -0.00000000e+00, -5.36817865e-10,  1.47841542e-08,
       -2.15848399e-10, -3.22466059e-16,  1.81756503e-19, -0.00000000e+00,
       -0.00000000e+00])

### Linear SVR

In [98]:
from sklearn.svm import LinearSVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_regression
regr = make_pipeline(StandardScaler(),LinearSVR(random_state=0, tol=1e-5))
pred = evaluate2(regr, train, test, train_y, real_return)
# evaluate2(model, train, test, train_y,test_y ,return_pred=True, version=2, return_auc=False, plot_auc=False):

          return         0
return  1.000000  0.034036
0       0.034036  1.000000
          0         0
0  1.000000  0.037487
0  0.037487  1.000000


### SVR

In [99]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.1))
pred = evaluate2(regr, train, test, train_y, real_return)
# from sklearn.svm import SVR
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import GridSearchCV

# # Define the pipeline
# regr = make_pipeline(StandardScaler(), SVR())

# # Define the hyperparameters to search over
# param_grid = {
#     'svr__C': [0.1, 1, 10],
#     'svr__gamma': [0.1, 1, 10],
#     'svr__epsilon': [0.1, 0.01, 0.001],
# }

# # Perform grid search with cross-validation
# grid_search = GridSearchCV(regr, param_grid, cv=5, n_jobs=-1)
# grid_search.fit(train, train_y)

# # Print the best hyperparameters and test score
# print("Best hyperparameters: ", grid_search.best_params_)
# print("Test score: ", grid_search.score(test, real_return))

          return         0
return  1.000000  0.145667
0       0.145667  1.000000
          0         0
0  1.000000 -0.002612
0 -0.002612  1.000000


### Gradient boosting

In [32]:
from sklearn import datasets, ensemble

for l in [0.99]:
    params = {
        "n_estimators": 200,
        "max_depth": 3,
        "min_samples_split": 8,
        "learning_rate": 0.01,
        "loss": "huber",
        "alpha": l,
    }

    reg = ensemble.GradientBoostingRegressor(**params)
    pred = evaluate2(reg, train, test, train_y, real_return)

          return         0
return  1.000000  0.170887
0       0.170887  1.000000
          0         0
0  1.000000  0.015523
0  0.015523  1.000000


### SGDBoosting

In [280]:
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
score_lst = []
# for i in [6508,36400]:
for i in range(1000,10000):
    for j in [1e-4,1e-5,1e-6]:
        reg = make_pipeline(StandardScaler(),SGDRegressor(max_iter=10000, tol=1e-6,penalty='l1',random_state=i))
        # pred = evaluate2(reg, train, test, train_y, real_return)
        pred, score = evaluate3(reg, train, test, train_y, real_return)
        print(score)
        score_lst.append(score)
print(max(score_lst), score_lst.index(max(score_lst)))

0.061560105033462806
0.061560105033462806
0.061560105033462806
0.06807174173976324
0.06807174173976324
0.06807174173976324
0.06199096481704358
0.06199096481704358
0.06199096481704358
0.05999080175204827
0.05999080175204827
0.05999080175204827
0.06292973318709269
0.06292973318709269
0.06292973318709269
0.050925137454654934
0.050925137454654934
0.050925137454654934
0.04871031743847376
0.04871031743847376
0.04871031743847376
0.06362077173358995
0.06362077173358995
0.06362077173358995
0.05045546512997368
0.05045546512997368
0.05045546512997368


KeyboardInterrupt: 

In [20]:
pred

array([[ 0.00083867],
       [ 0.00175657],
       [ 0.00125077],
       ...,
       [-0.00226737],
       [-0.00159325],
       [-0.00088492]])

### Random Forest

In [36]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_regression
regr = RandomForestRegressor(max_depth=7, random_state=6508, max_features=2, min_samples_leaf=4, min_samples_split=8, n_estimators=300)
pred = evaluate2(regr, train, test, train_y, real_return)

# Create the parameter grid based on the results of random search 
# param_grid = {
#     'bootstrap': [True],
#     'max_depth': [7, 8, 9, 10],
#     'max_features': [2, 3],
#     'min_samples_leaf': [3, 4, 5],
#     'min_samples_split': [8, 10, 12],
#     'n_estimators': [100, 200, 300, 1000]
# }

# # Create a based model
# rf = RandomForestRegressor()
# # Instantiate the grid search model
# grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
#                           cv = 3, n_jobs = -1, verbose = 2)
# # Fit the grid search to the data
# grid_search.fit(train, train_y)
# grid_search.best_params_

          return         0
return  1.000000  0.231504
0       0.231504  1.000000
          0         0
0  1.000000  0.032785
0  0.032785  1.000000


## Submission

In [229]:
submit_arr = np.array(pred_com.iloc[:,1])
submit_arr

array([-6.18817306e-03, -5.03549075e-03, -5.15205865e-03, -6.13767062e-03,
       -6.17336116e-03, -7.65731977e-03, -7.08189447e-03, -2.95779751e-03,
       -4.87115217e-03, -8.03174853e-03, -6.96429590e-03, -8.36665442e-03,
       -6.57850873e-03, -7.49549072e-03, -6.15497027e-03, -7.59540658e-03,
       -6.30651724e-03, -1.26222106e-03, -7.57796735e-03, -6.55519578e-03,
       -8.02599537e-03,  2.52904472e-02,  3.99932029e-02, -4.14283803e-04,
       -5.57382300e-03, -6.46932151e-03, -7.43853512e-03, -7.74659080e-03,
       -8.14983805e-03, -6.95487074e-03, -4.34695932e-03, -5.42129580e-03,
       -5.09428038e-03, -2.47650310e-03, -5.72436558e-03, -6.67787559e-03,
       -7.23372951e-03, -8.68383356e-03, -4.66569681e-03, -4.18243040e-03,
       -7.68435363e-03, -7.69498664e-03, -2.93341994e-03,  2.38353844e-03,
       -3.99585507e-03, -6.09223077e-03, -3.26634000e-03, -4.88251374e-03,
       -5.95786986e-03, -5.28317267e-03, -6.16077183e-03, -6.51464957e-03,
       -5.03345292e-03, -

In [230]:
submit(submit_arr)